In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

In [2]:
directory_in = './file/2.oem'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：731



,Part Number,[UAC] Part Number,[UAC] OEM,[UAC] Picture,[UAC] Url,[UAC] Json_Src,[UAC] Json_Note
0,HA10018C,HA 10018C,1510750;1531327;707048;F3ZZ19D734BA;55664;4811...,,https://uacparts.com/Catalog/SearchIndex?searc...,"{""0"": ""https://www.uacparts.com/photo/WEBP/HA/...","{""0"": ""Accumulator with Hose Assembly""}"
1,HA10020C,HA 10020C,1531251;51535;10602641;T55308;F7DZ17C836EA;F7D...,,https://uacparts.com/Catalog/SearchIndex?searc...,"{""0"": ""https://www.uacparts.com/photo/WEBP/HA/...","{""0"": ""Accumulator with Hose Assembly""}"
2,HA10021C,HA 10021C,1510188;1510905;11608011;T83015;707006;2C2419C...,,https://uacparts.com/Catalog/SearchIndex?searc...,"{""0"": ""https://www.uacparts.com/photo/WEBP/HA/...","{""0"": ""Accumulator with Hose Assembly""}"
3,HA10022C,HA 10022C,1510189;1510910;51500;10602201;707008;2C2Z19C8...,,https://uacparts.com/Catalog/SearchIndex?searc...,"{""0"": ""https://www.uacparts.com/photo/WEBP/HA/...","{""0"": ""Accumulator with Hose Assembly""}"
4,HA10029C,HA 10029C,1510733;151777;1530874;T33751;T33751;707017;AD...,,https://uacparts.com/Catalog/SearchIndex?searc...,"{""0"": ""https://www.uacparts.com/photo/WEBP/HA/...","{""0"": ""Accumulator with Hose Assembly""}"
...,...,...,...,...,...,...,...
726,RD9993C,RD 9993C,1510051;151237;80351SX0961;3162;71271;11605211...,,https://uacparts.com/Catalog/SearchIndex?searc...,"{""0"": ""https://www.uacparts.com/photo/WEBP/RD/...","{""0"": ""Drier Pad Mount""}"
727,RD9996C,RD 9996C,1510233;80351SN7003;80351SP0013;71225;11610681...,,https://uacparts.com/Catalog/SearchIndex?searc...,"{""0"": ""https://www.uacparts.com/photo/WEBP/RD/...","{""0"": ""Drier Pad Mount""}"
728,RD9997C,RD 9997C,1510063;64538353039;64538372977;64538391313;70...,,https://uacparts.com/Catalog/SearchIndex?searc...,"{""0"": ""https://www.uacparts.com/photo/WEBP/RD/...","{""0"": ""Drier Pad Mount""}"
729,RD9998C,RD 9998C,PC400;8309998;F859,,https://uacparts.com/Catalog/SearchIndex?searc...,"{""0"": ""https://www.uacparts.com/photo/WEBP/RD/...","{""0"": ""Drier""}"


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 Part Number                                                   HA10018C
 [UAC] Part Number                                            HA 10018C
 [UAC] OEM            1510750;1531327;707048;F3ZZ19D734BA;55664;4811...
 [UAC] Picture                                                         
 [UAC] Url            https://uacparts.com/Catalog/SearchIndex?searc...
 [UAC] Json_Src       {"0": "https://www.uacparts.com/photo/WEBP/HA/...
 [UAC] Json_Note                {"0": "Accumulator with Hose Assembly"}
 Name: 0, dtype: object)

In [5]:
import time

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_check = crawler_series['[UAC] Url']

        resp = requests.get(f'https://uacparts.com/Catalog/Specs?part={crawler_series['[UAC] Part Number']}',
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            if html.xpath('//form[@action="/Catalog/Specs"]'):
                with open('./spec.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

print(soup.prettify())

1
<html>
 <body>
  <div>
   <br/>
  </div>
  <form action="/Catalog/Specs" method="post">
   <div>
    <table class="awe-grid table">
    </table>
   </div>
  </form>
 </body>
</html>



In [6]:
list_tr, dict_count = html.xpath('//tr[@class="awe-row"]'), {}
for tr in list_tr:
    key = tr.xpath('./td/text()')[0].strip()
    if key in dict_count:
        dict_count[key] += 1
    else:
        dict_count[key] = 1

dict_spec, dict_no = {}, {}
for i, tr in enumerate(list_tr):
    key = tr.xpath('./td/text()')[0].strip()
    if dict_count[key] == 1:
        dict_spec[str(i)] = {key: tr.xpath('./td/text()')[1].strip()}
    else:
        if key in dict_no:
            dict_no[key] += 1
        else:
            dict_no[key] = 1
        dict_spec[str(i)] = {f'{key}-{dict_no[key]}': tr.xpath('./td/text()')[1].strip()}

dict_spec

{}

In [7]:
list_dict_correct.append({'Part Number': crawler_series['Part Number'],
                          '[UAC] Part Number': crawler_series['[UAC] Part Number'],
                          '[UAC] OEM': crawler_series['[UAC] OEM'],
                          '[UAC] Picture': crawler_series['[UAC] Picture'],
                          '[UAC] Url': crawler_series['[UAC] Url'],
                          '[UAC] Json_Src': crawler_series['[UAC] Json_Src'],
                          '[UAC] Json_Note': crawler_series['[UAC] Json_Note'],
                          '[UAC] Json_Spec': json.dumps(dict_spec)})

list_dict_correct

[{'Part Number': 'HA10018C',
  '[UAC] Part Number': 'HA 10018C',
  '[UAC] OEM': '1510750;1531327;707048;F3ZZ19D734BA;55664;4811402;YF1703;200758;TEM200758;3656;H6196;H6306;520058',
  '[UAC] Picture': '',
  '[UAC] Url': 'https://uacparts.com/Catalog/SearchIndex?search=HA10018C',
  '[UAC] Json_Src': '{"0": "https://www.uacparts.com/photo/WEBP/HA/HA 10018C_1_562.webp", "1": "https://www.uacparts.com/photo/WEBP/HA/HA 10018C_4_562.webp"}',
  '[UAC] Json_Note': '{"0": "Accumulator with Hose Assembly"}',
  '[UAC] Json_Spec': '{}'}]

In [8]:
crawler_status = True

crawler_status

True

In [9]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[Part Number                                                   HA10018C
 [UAC] Part Number                                            HA 10018C
 [UAC] OEM            1510750;1531327;707048;F3ZZ19D734BA;55664;4811...
 [UAC] Picture                                                         
 [UAC] Url            https://uacparts.com/Catalog/SearchIndex?searc...
 [UAC] Json_Src       {"0": "https://www.uacparts.com/photo/WEBP/HA/...
 [UAC] Json_Note                {"0": "Accumulator with Hose Assembly"}
 Check_Url            https://uacparts.com/Catalog/SearchIndex?searc...
 Name: 0, dtype: object]

In [10]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Part Number'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_spec.xlsx', index=False)

df_correct

,Part Number,[UAC] Part Number,[UAC] OEM,[UAC] Picture,[UAC] Url,[UAC] Json_Src,[UAC] Json_Note,[UAC] Json_Spec
0,HA10018C,HA 10018C,1510750;1531327;707048;F3ZZ19D734BA;55664;4811...,,https://uacparts.com/Catalog/SearchIndex?searc...,"{""0"": ""https://www.uacparts.com/photo/WEBP/HA/...","{""0"": ""Accumulator with Hose Assembly""}",{}


In [11]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Part Number'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_spec_error.xlsx', index=False)

df_error

,Part Number,[UAC] Part Number,[UAC] OEM,[UAC] Picture,[UAC] Url,[UAC] Json_Src,[UAC] Json_Note,Check_Url
0,HA10018C,HA 10018C,1510750;1531327;707048;F3ZZ19D734BA;55664;4811...,,https://uacparts.com/Catalog/SearchIndex?searc...,"{""0"": ""https://www.uacparts.com/photo/WEBP/HA/...","{""0"": ""Accumulator with Hose Assembly""}",https://uacparts.com/Catalog/SearchIndex?searc...
